In [34]:
import os
import numpy as np
import IPython
import copy
from shutil import copyfile

import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets import base


import sys
sys.dont_write_bytecode=True

PACKAGE_PARENT = '../'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))

from influence.inceptionModel import BinaryInceptionModel
from influence.logisticRegressionWithLBFGS import LogisticRegressionWithLBFGS
import influence.experiments
from influence.dataset import DataSet
# from influence.dataset_poisoning import iterative_attack, select_examples_to_attack, get_projection_to_box_around_orig_point, generate_inception_features
from influence.iter_attack import iterative_attack, select_examples_to_attack, get_projection_to_box_around_orig_point, generate_inception_features
from influence.Progress import *

from load_animals import *

from skimage import io

%load_ext autoreload
%autoreload 2

from data_poisoning import data_poisoning

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
data_selected = ['dog', 'fish']
num_train_ex_per_class, num_test_ex_per_class = 900, 300
use_IF = True
target_test_idx = [45]
num_to_perterb = 1

In [36]:
# 1. Poison the dataset 
target_idx, poisoned_image = data_poisoning(['dog', 'fish'], 900, 300, True, target_test_idx, 1)

Loading animals from disk...
../data/dataset_dog-fish_train-900_test-300.npz
*** Full:
self.logits Tensor("Shape:0", shape=(2,), dtype=int32)


[genericNeuralNet.py:191 -   get_vec_to_list_fn() ] Total number of parameters: 2048


wrong_labels_bool Tensor("Shape_2:0", shape=(2,), dtype=int32)
logits Tensor("Shape_3:0", shape=(2,), dtype=int32)
inception_features:  Tensor("flatten/Reshape:0", shape=(?, ?), dtype=float32)
x_poison_features:  Tensor("Gather:0", shape=(1, ?), dtype=float32)
t_target_features:  Tensor("Gather_1:0", shape=(1, ?), dtype=float32)
Lp:  Tensor("norm/Squeeze:0", shape=(), dtype=float32)
LP_gradient Tensor("strided_slice_1:0", shape=(268203,), dtype=float32)
*** Top:
self.logits Tensor("Shape:0", shape=(2,), dtype=int32)


[genericNeuralNet.py:191 -   get_vec_to_list_fn() ] Total number of parameters: 2048


wrong_labels_bool Tensor("Shape_2:0", shape=(2,), dtype=int32)
logits Tensor("Shape_3:0", shape=(2,), dtype=int32)
Using normal model


...

[iter_attack.py:160 -     iterative_attack() ] Attach_iter 99 perturbation shape: (1, 268203), perturbation: [[ 0.0405162   1.58749163 -1.87807417 ... -3.19072056 -0.71753657
   3.30716562]]


         Current function value: -37746.640625
         Iterations: 10
         Function evaluations: 153
         Gradient evaluations: 146
         Hessian evaluations: 55
Inverse HVP took 3.69466900826 sec
Loaded inverse HVP from output/poisoning_900_300_inception_wd-0.001-test-[45].npz
Inverse HVP took 0.000514984130859 sec
Entering the for loop
(1, 268203)
1 1


[genericNeuralNet.py:328 -     print_model_eval() ] Train loss (w reg) on all data: 0.013667375
[genericNeuralNet.py:329 -     print_model_eval() ] Train loss (w/o reg) on all data: 0.004501966
[genericNeuralNet.py:331 -     print_model_eval() ] Test loss (w/o reg) on all data: 0.04704576
[genericNeuralNet.py:332 -     print_model_eval() ] Train acc on all data:  1.0
[genericNeuralNet.py:333 -     print_model_eval() ] Test acc on all data:   0.985
[genericNeuralNet.py:335 -     print_model_eval() ] Norm of the mean of gradients: 0.0052796495
[genericNeuralNet.py:336 -     print_model_eval() ] Norm of the params: 4.2814503


Shape (1, 2048)
Using normal model
LBFGS training took [21] iter.
After training with LBFGS: 


[iter_attack.py:204 -     iterative_attack() ] Test_idx: 45 Test pred (full): [[0.03766986 0.96233016]]
[iter_attack.py:205 -     iterative_attack() ] ---------------------
[iter_attack.py:210 -     iterative_attack() ] Test_idx: 45 Test pred (top): [[0.03766986 0.96233016]]


In [37]:
# 2. Generate all the inception features

# Find the cleaned dataset
data_sets = load_animals(
                    num_train_ex_per_class=num_train_ex_per_class, 
                    num_test_ex_per_class=num_test_ex_per_class,
                    classes=data_selected)

# Replace the target training point with the poisoned one
poisoned_dataset_x = np.copy(data_sets.train.x)
poisoned_dataset_x[target_idx] = poisoned_image # data_sets.test.x[15] 
poisoned_dataset_labels = data_sets.train.labels

poisoned_dataset = DataSet(poisoned_dataset_x, poisoned_dataset_labels)

# Create the DateSet that only contain the target test point
target_test_dataset = DataSet(data_sets.test.x[target_test_idx], data_sets.test.labels[target_test_idx])

# Create the clean and poisoned training inception features
# and the inception feature of the target test point
img_side = 299
num_channels = 3
 
initial_learning_rate = 0.001 
keep_probs = None
decay_epochs = [1000, 10000]

weight_decay = 0.001

num_classes = len(data_selected)
max_lbfgs_iter = 1000
batch_size = 50

dataset_name = 'transferbility_%s_%s' % (num_train_ex_per_class, num_test_ex_per_class)

full_graph = tf.Graph()
with full_graph.as_default():
    full_model_name = '%s_inception_wd-%s' % (dataset_name, weight_decay)
    full_model = BinaryInceptionModel(
        img_side=img_side,
        num_channels=num_channels,
        weight_decay=weight_decay,
        num_classes=num_classes, 
        batch_size=batch_size,
        data_sets=data_sets,
        initial_learning_rate=initial_learning_rate,
        keep_probs=keep_probs,
        decay_epochs=decay_epochs,
        mini_batch=True,
        train_dir='output',
        log_dir='log',
        model_name=full_model_name)
    
    clean_inception_features = full_model.generate_inception_features(data_sets.train, None)
    poisoned_inception_features = full_model.generate_inception_features(poisoned_dataset, None)
    target_test_inception_features = full_model.generate_inception_features(target_test_dataset, None)

Loading animals from disk...
../data/dataset_dog-fish_train-900_test-300.npz
self.logits Tensor("Shape:0", shape=(2,), dtype=int32)


[genericNeuralNet.py:191 -   get_vec_to_list_fn() ] Total number of parameters: 2048


wrong_labels_bool Tensor("Shape_2:0", shape=(2,), dtype=int32)
logits Tensor("Shape_3:0", shape=(2,), dtype=int32)
inception_features:  Tensor("flatten/Reshape:0", shape=(?, ?), dtype=float32)
x_poison_features:  Tensor("Gather:0", shape=(1, ?), dtype=float32)
t_target_features:  Tensor("Gather_1:0", shape=(1, ?), dtype=float32)
Lp:  Tensor("norm/Squeeze:0", shape=(), dtype=float32)
LP_gradient Tensor("strided_slice_1:0", shape=(268203,), dtype=float32)


In [40]:
# 3. Feed to SVM and check if flips or not 
#    To compare, also show the results of Log Reg

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

C = 1.0 / (len(data_sets.train.x) * weight_decay)  
if num_classes == 2:
    log_reg_model = LogisticRegression(
                C=C,
                tol=1e-8,
                fit_intercept=False, 
                solver='lbfgs',
                warm_start=True, #True
                max_iter=max_lbfgs_iter)
else:
    log_reg_model = LogisticRegression(
                C=C,
                tol=1e-8,
                fit_intercept=False, 
                solver='lbfgs',
                multi_class='multinomial',
                warm_start=True, #True
                max_iter=max_lbfgs_iter) 
    
svc_model = SVC(
                C=C,
                kernel='linear'
                )

print("True label of target test point: ", data_sets.test.labels[target_test_idx])
log_reg_model.fit(clean_inception_features, data_sets.train.labels)
print("Inital prediction (Log Reg): ", log_reg_model.predict(target_test_inception_features))
log_reg_model.fit(poisoned_inception_features, data_sets.train.labels)
print("Final prediction (Log Reg): ", log_reg_model.predict(target_test_inception_features))

svc_model.fit(clean_inception_features, data_sets.train.labels)
print("Inital prediction (SVM): ", svc_model.predict(target_test_inception_features))
svc_model.fit(poisoned_inception_features, data_sets.train.labels)
print("Final prediction (SVM): ", svc_model.predict(target_test_inception_features))

('True label of target test point: ', array([0.]))
('Inital prediction (Log Reg): ', array([0.]))
('Final prediction (Log Reg): ', array([1.]))
('Inital prediction (SVM): ', array([0.]))
('Final prediction (SVM): ', array([1.]))


[0.]
